# Install TensorFlow and OpenCV

**bold text**

In [ ]:
!pip install opencv-python tensorflow tensorflow_hub


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained model from TensorFlow Hub
model = hub.load('https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2')

# Function to perform object detection on a frame
def detect_objects(frame, model):
    # Preprocess the frame
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Perform object detection
    detections = model(input_tensor)

    return detections

# Function to draw bounding boxes around detected objects
def draw_boxes(frame, detections):
    for i in range(len(detections['detection_scores'][0])):
        score = detections['detection_scores'][0][i].numpy()
        if score > 0.5:  # Confidence threshold
            bbox = [int(coord) for coord in detections['detection_boxes'][0][i].numpy() * np.array([frame.shape[0], frame.shape[1], frame.shape[0], frame.shape[1]])]
            cv2.rectangle(frame, (bbox[1], bbox[0]), (bbox[3], bbox[2]), (0, 255, 0), 2)
            cv2.putText(frame, f"{score:.2f}", (bbox[1], bbox[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Capture video from the camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    detections = detect_objects(frame, model)

    # Draw bounding boxes around detected objects
    draw_boxes(frame, detections)

    # Display the frame
    cv2.imshow('Object Detection', frame)

    # Exit on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained model from TensorFlow Hub
model = hub.load('https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2')

# Function to initialize camera
def initialize_camera():
    cap = cv2.VideoCapture(0)  # Use 0 for default camera
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return None
    return cap

# Initialize camera
cap = initialize_camera()
if cap is None:
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Perform object detection
    converted_img = tf.image.convert_image_dtype(frame, tf.float32)[tf.newaxis, ...]
    result = model(converted_img)

    # Draw bounding boxes on the frame
    for i in range(len(result['detection_scores'][0])):
        score = result['detection_scores'][0][i]
        bbox = [float(v) for v in result['detection_boxes'][0][i]]
        if score > 0.5:  # Filter out low-confidence detections
            ymin, xmin, ymax, xmax = bbox
            xmin = int(xmin * frame.shape[1])
            xmax = int(xmax * frame.shape[1])
            ymin = int(ymin * frame.shape[0])
            ymax = int(ymax * frame.shape[0])
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(frame, f'{score:.2f}', (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Object Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


Error: Could not open camera.


AttributeError: 'NoneType' object has no attribute 'read'

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained model from TensorFlow Hub
model = hub.load('https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2')

# Function to initialize camera
def initialize_camera():
    cap = cv2.VideoCapture(0)  # Use 0 for default camera
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return None
    print("Camera initialized successfully.")
    return cap

# Initialize camera
cap = initialize_camera()
if cap is None:
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Perform object detection
    converted_img = tf.image.convert_image_dtype(frame, tf.float32)[tf.newaxis, ...]
    result = model(converted_img)

    # Draw bounding boxes on the frame
    for i in range(len(result['detection_scores'][0])):
        score = result['detection_scores'][0][i]
        bbox = [float(v) for v in result['detection_boxes'][0][i]]
        if score > 0.5:  # Filter out low-confidence detections
            ymin, xmin, ymax, xmax = bbox
            xmin = int(xmin * frame.shape[1])
            xmax = int(xmax * frame.shape[1])
            ymin = int(ymin * frame.shape[0])
            ymax = int(ymax * frame.shape[0])
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(frame, f'{score:.2f}', (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Object Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Install TensorFlow and OpenCV
!pip install tensorflow opencv-python-headless !pip install opencv-python tensorflow tensorflow_hub



import the necessary libraries and set up our object detection model.

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained model from TensorFlow Hub
model = hub.load('https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2')


 define a function to perform object detection on a given image

In [ ]:
def detect_objects(image):
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = model(input_tensor)

    # Convert detections to numpy arrays
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Detection classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    return detections


 capture images from the camera in real-time and perform object detection on them

In [ ]:
# Function to capture images from camera
def capture_camera():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to capture frame")
            break

        # Perform object detection on the captured frame
        detections = detect_objects(frame)

        # Draw bounding boxes around detected objects
        for i in range(len(detections['detection_classes'])):
            class_id = detections['detection_classes'][i]
            box = detections['detection_boxes'][i]
            confidence = detections['detection_scores'][i]

            if confidence > 0.5:  # Filter out detections with low confidence
                h, w, _ = frame.shape
                ymin, xmin, ymax, xmax = box
                xmin = int(xmin * w)
                xmax = int(xmax * w)
                ymin = int(ymin * h)
                ymax = int(ymax * h)

                # Draw bounding box and label
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(frame, f'Class: {class_id}, Confidence: {confidence}', (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the frame with detections
        cv2.imshow('Object Detection', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the function to start capturing from the camera
capture_camera()
